In [ ]:
import os
import requests
from io import BytesIO
from PIL import Image
import base64
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]

In [4]:
with open("sample.txt", "r") as f:
    sample = f.read()

In [72]:
with open("sample-2.txt", "r") as f:
    sample_2 = f.read()

In [5]:
sample[:100]

'Summary of the article and thread (brief)\nArticle: OpenAI announces itself (Dec 2015) as a non‑profi'

In [51]:
from pathlib import Path

def make_unique_dir(base: str, suffix_format: str = "{}"):
    path = Path(base)
    counter = 1
    while path.exists():
        path = Path(f"{base}_{suffix_format.format(counter)}")
        counter += 1
    path.mkdir(parents=True)
    return path



In [53]:
def generate_image(prompt: str, model: str) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "modalities": ["image", "text"],
        "image_config": {
            "aspect_ratio": "16:9"
        }
    }

    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    if result.get("choices"):
        message = result["choices"][0]["message"]
        if message.get("images"):
            dir_path = make_unique_dir("slides", suffix_format="{:02d}")
            for idx, image in enumerate(message["images"]):
                image_url = image["image_url"]["url"]
                print(f"Generated image: {image_url[:50]}...")
   
                image_data = image_url.split(",", 1)[1]
                image_data = base64.b64decode(image_data)
                with BytesIO(image_data) as image_buffer:
                    image = Image.open(image_buffer)
                    image.save(f"{dir_path}/output_{idx}.png", format="PNG")
    return result



In [54]:
def generate_text(prompt: str, model: str) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]      
    }

    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    return result

In [55]:
from langgraph.graph import StateGraph, START, END, add_messages

In [56]:
from typing import Annotated, TypedDict
from langchain.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage

In [57]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    prompt: str

In [58]:
PLANNER_PROMPT = """
    You are a planner to make slides.
    You are given texts and you need to plan for making slides.
    1. figure out the main topic of the texts.
    2. decompose the texts into with their semantic meaning.
    3. make plans to generate slide images for each semantic meaning.
    Note: A slide has only one sub topic.
    Texts: {texts}
   
    """

In [59]:
GENERATE_PROMPT = """
    You are a slide maker.
    You are given a plan and you need to generate slide images.
    Plan: {plan}
   
    """

In [66]:
def planner(state: GraphState) -> GraphState:
    messages = state["messages"][-1].content
    res = generate_text(PLANNER_PROMPT.format(texts=messages), model="google/gemini-3-pro-preview")
    return {"messages": [AIMessage(content=res["choices"][0]["message"]["content"])]}

def generate_slide(state: GraphState) -> GraphState:
    plan = state["messages"][-1].content
    res = generate_image(GENERATE_PROMPT.format(plan=plan), model="google/gemini-2.5-flash-image")
    return {"messages": [AIMessage(content=res["choices"][0]["message"]["content"])]}


In [67]:
graph = StateGraph(GraphState)
graph.add_node("planner", planner)
graph.add_node("generate_slide", generate_slide)
graph.add_edge(START, "planner")
graph.add_edge("planner", "generate_slide")
graph.add_edge("generate_slide", END)

In [68]:
from langgraph.checkpoint.memory import MemorySaver

In [69]:
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

In [70]:
config={"configurable": {"thread_id": "1"}}

In [73]:
for chunk in app.stream({"messages": [{"role": "user", "content": sample_2}]}, stream_mode="updates", config=config):
    print(chunk)


{'planner': {'messages': [AIMessage(content='Here is the slide planning based on the provided text.\n\n### 1. Main Topic\n**The Historic Pivot: How SpaceX’s Orbcomm-2 Landing Defined a Decade of Spaceflight (2015–2025)**\n\n---\n\n### 2. Decomposition of Semantic Meaning\nThe text can be broken down into the following narrative arc:\n*   **The Catalyst:** The specific event (Orbcomm-2 mission) and the immediate emotional/technical reaction in 2015.\n*   **The Evolution:** How reuse went from a "stunt" to a boring routine (Block 5, reliability).\n*   **The Market Shift:** The collapse of traditional competitors and SpaceX\'s resulting dominance.\n*   **The "Why":** Starlink as the economic engine utilized by cheap launch costs.\n*   **The Comparison:** The "SpaceX vs. Blue Origin" debate and why the "Welcome to the Club" comment aged poorly.\n*   **The Verdict:** A look back at who predicted the future correctly (systems engineering vs. skepticism).\n\n---\n\n### 3. Slide Plan\n\n#### S